In [ ]:
import numpy
import scipy.integrate
import pyfabm

%matplotlib widget
from matplotlib import pyplot

In [ ]:
# Create model (loads fabm.yaml)
model = pyfabm.Model('../extern/fabm-medusa/testcases/fabm-medusa-original.yaml')

column_depth = 50

# Configure the environment
# Note: the set of environmental dependencies depends on the loaded biogeochemical model.
model.cell_thickness = column_depth
model.findDependency('temperature').value = 15.
model.findDependency('practical_salinity').value = 35.
model.findDependency('surface_downwelling_shortwave_flux').value = 50.
model.findDependency('density').value = 1020
model.findDependency('cell_thickness').value = column_depth
model.findDependency('depth').value = 0.5 * column_depth
model.findDependency('pressure').value = 0.5 * column_depth
model.findDependency('ice_area_fraction').value = 0
model.findDependency('wind_speed').value = 1.

# Verify the model is ready to be used
assert model.checkReady(), 'One or more model dependencies have not been fulfilled.'

In [ ]:
# Time derivative
def dy(y, t0):
    model.state[:] = y
    return model.getRates()

# Time-integrate over 200 days (note: FABM's internal time unit is seconds!)
t = numpy.linspace(0, 200., 1000)
y = scipy.integrate.odeint(dy, model.state, t*86400)

In [ ]:
# Plot results
fig, ax = pyplot.subplots()
for i, variable in enumerate(model.state_variables):
    if variable.name in ('pelagic/PHN', 'pelagic/PHD', 'pelagic/ZME', 'pelagic/ZMI'):
        ax.plot(t, y[:,i], label=variable.long_name)
ax.legend()
ax.grid()
ax.set_xlabel('time (d)');